In [1]:
import pandas as pd
import sqlite3
import glob
import os

In [8]:
eventTypeTableFilenames = [fn for fn in glob.glob('event_*.csv') 
                           if not fn == "event_map_type.csv" and not fn == "event_object.csv"]
objectTypeTableFilenames =  [fn for fn in glob.glob('object_*.csv') 
                             if not fn == "object_map_type.csv" and not fn == "object_object.csv"]

TABLES = dict()

TABLES["event"] = pd.read_csv("event.csv", sep=";")
TABLES["event_map_type"] = pd.read_csv("event_map_type.csv", sep=";")
TABLES["event_object"] = pd.read_csv("event_object.csv", sep=";")
TABLES["object"] = pd.read_csv("object.csv", sep=";")
TABLES["object_object"] = pd.read_csv("object_object.csv", sep=";")
TABLES["object_map_type"] = pd.read_csv("object_map_type.csv", sep=";")

for fn in eventTypeTableFilenames:
    table_name = fn.split(".")[0]
    table = pd.read_csv(fn, sep=";")
    TABLES[table_name] = table
    
for fn in objectTypeTableFilenames:
    table_name = fn.split(".")[0]
    table = pd.read_csv(fn, sep=";")
    TABLES[table_name] = table

In [10]:
sql_path = "IoT_CargoPickup.sqlite"
if os.path.exists(sql_path):
    os.remove(sql_path)

In [11]:
conn = sqlite3.connect(sql_path)
for tn, df in TABLES.items():
    df.to_sql(tn, conn, index=False)
conn.close()

In [12]:
# Connect to your SQLite database
conn = sqlite3.connect("IoT_CargoPickup.sqlite")
cursor = conn.cursor()

# Create a new table without duplicates
cursor.execute('''CREATE TABLE IF NOT EXISTS object_object_no_duplicates AS
                  SELECT DISTINCT * FROM object_object''')

# Drop the original table
cursor.execute('DROP TABLE IF EXISTS object_object')

# Rename the new table to the original table name
cursor.execute('ALTER TABLE object_object_no_duplicates RENAME TO object_object')

# Commit changes and close the connection
conn.commit()
conn.close()

In [13]:
import pm4py

ocel = pm4py.read_ocel2_sqlite("IoT_CargoPickup.sqlite")
ocel

C:\Users\n9810200\Anaconda3\lib\site-packages\pm4py\objects\ocel\importer\sqlite\variants\ocel20.py:70: UserWarning: There are unsatisfied OCEL 2.0 constraints in the given relational database: ['const_2_existence_object_type_tables_map_obj_types', 'const_3_existence_event_type_tables_map_ev_types', 'const_14_primary_key_object_event_map_type_tables', 'const_15_primary_key_object_event_tables', 'const_16_primary_key_event_object_table', 'const_17_primary_key_object_object_table', 'const_18_primary_key_event_type_spec_tables', 'const_19_foreign_key_event', 'const_20_foreign_key_object', 'const_21_foreign_key_event_object', 'const_22_foreign_key_object_object', 'const_23_foreign_key_event_type_specific', 'const_24_foreign_key_object_type_specific']
  warnings.warn("There are unsatisfied OCEL 2.0 constraints in the given relational database: "+str(unsatisfied))
C:\Users\n9810200\Anaconda3\lib\site-packages\pm4py\objects\log\util\dataframe_utils.py:183: SettingWithCopyWarning: 
A value is 

Object-Centric Event Log (number of events: 2110, number of objects: 79, number of activities: 12, number of object types: 4, events-objects relationships: 2120)
Activities occurrences: {'Assign Trucks': 459, 'Enter the port': 459, 'Weigh the Empty Truck': 459, 'Fail to Weigh': 415, 'Arrive at the Silo': 44, 'Determine the Continuance of the Pickup': 44, 'Load Truck': 44, 'Weigh the Loaded Truck': 44, 'Evaluate the Truck Exit': 44, 'Input the Tally Sheet': 44, 'Print the Weighing Ticket': 44, 'Lodge Pickup Plan': 10}
Object types occurrences (number of objects): {'Truck': 50, 'Cargo': 10, 'Pickup Plan': 10, 'Silo': 9}
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.

In [14]:
PP_lattened_log = pm4py.ocel_flattening(ocel, "Pickup Plan")

In [15]:
PP_lattened_log

,ocel:eid,time:timestamp,Silo ID,Truck ID,concept:name,case:concept:name,case:Cargo Type,case:Cargo stock weight(scheduled),case:Silo ID,case:Cargo ID,...,case:Grain Temperature,case:RFID No,case:Pickup Plan ID,case:LPT,case:Axles,case:Scheduled Pickup Weight,case:Truck Status,case:Truck Weight,case:Is_normal,case:ocel:type
0,Lodge_Pcp2,2024-04-29 07:33:49+00:00,NaN,NaN,Lodge Pickup Plan,Pcp2,NaN,NaN,NaN,Cr16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pickup Plan
1,Lodge_Pcp7,2024-04-29 07:34:21+00:00,NaN,NaN,Lodge Pickup Plan,Pcp7,NaN,NaN,NaN,Cr5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pickup Plan
2,Lodge_Pcp10,2024-04-29 07:40:30+00:00,NaN,NaN,Lodge Pickup Plan,Pcp10,NaN,NaN,NaN,Cr1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pickup Plan
3,Lodge_Pcp4,2024-04-29 08:03:32+00:00,NaN,NaN,Lodge Pickup Plan,Pcp4,NaN,NaN,NaN,Cr4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pickup Plan
4,Lodge_Pcp5,2024-04-29 08:03:41+00:00,NaN,NaN,Lodge Pickup Plan,Pcp5,NaN,NaN,NaN,Cr8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pickup Plan
5,Lodge_Pcp9,2024-04-29 08:20:41+00:00,NaN,NaN,Lodge Pickup Plan,Pcp9,NaN,NaN,NaN,Cr19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pickup Plan
6,Lodge_Pcp8,2024-04-29 08:31:20+00:00,NaN,NaN,Lodge Pickup Plan,Pcp8,NaN,NaN,NaN,Cr6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pickup Plan
7,Lodge_Pcp6,2024-04-29 09:00:11+00:00,NaN,NaN,Lodge Pickup Plan,Pcp6,NaN,NaN,NaN,Cr15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pickup Plan
8,Lodge_Pcp3,2024-04-29 09:01:50+00:00,NaN,NaN,Lodge Pickup Plan,Pcp3,NaN,NaN,NaN,Cr10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pickup Plan
9,Lodge_Pcp1,2024-04-29 09:09:07+00:00,NaN,NaN,Lodge Pickup Plan,Pcp1,NaN,NaN,NaN,Cr12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pickup Plan


In [16]:
PP_lattened_log.to_csv('port_pickup_iot_xes.csv', index = False)

In [17]:
Tr_lattened_log = pm4py.ocel_flattening(ocel, "Truck")
Tr_lattened_log.to_csv('port_truck_iot_xes.csv', index = False)